## Week 2 Homework

> In case you don't get one option exactly, select the closest one 

For the homework, we'll be working with the _green_ taxi dataset located here:

`https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/green/download`

In [1]:
url_download = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/tag/green/download'

### Assignment

The goal will be to construct an ETL pipeline that loads the data, performs some transformations, and writes the data to a database (and Google Cloud!).

- Create a new pipeline, call it `green_taxi_etl`
- Add a data loader block and use Pandas to read data for the final quarter of 2020 (months `10`, `11`, `12`).
  - You can use the same datatypes and date parsing methods shown in the course.
  - `BONUS`: load the final three months using a for loop and `pd.concat`
- Add a transformer block and perform the following:
  - Remove rows where the passenger count is equal to 0 _or_ the trip distance is equal to zero.
  - Create a new column `lpep_pickup_date` by converting `lpep_pickup_datetime` to a date.
  - Rename columns in Camel Case to Snake Case, e.g. `VendorID` to `vendor_id`.
  - Add three assertions:
    - `vendor_id` is one of the existing values in the column (currently)
    - `passenger_count` is greater than 0
    - `trip_distance` is greater than 0
- Using a Postgres data exporter (SQL or Python), write the dataset to a table called `green_taxi` in a schema `mage`. Replace the table if it already exists.
- Write your data as Parquet files to a bucket in GCP, partioned by `lpep_pickup_date`. Use the `pyarrow` library!

- Schedule your pipeline to run daily at 5AM UTC.

```bash
cd homework/02-workflow-orchestration
docker compose up
```

Open [http://localhost:6789](http://localhost:6789)

Create pipeline
1. `+ New Pipeline > Standard (batch)`
1. `Edit pipeline settinngs` rename to `green_taxi_etl`
1. Add tag `dezoomcamp_homework_2`
1. `Save pipeline settings`

Edit pipeline
1. Select `Edit pipeline` from sidebar
1. `+ Data loader > Python > Generic (no template)` rename to `read_final_quarter_2020` -> `Save and add`

Edit data loader
```python
@data_loader
def load_data(*args, **kwargs):
    """
    Template code for loading data from any source.

    Returns:
        Anything (e.g. data frame, dictionary, array, int, str, etc.)
    """
    url_download = 'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/'
    year = 2020
    months = [10, 11, 12]

    #green_tripdata_{year}-{month}.csv.gz

    # reduces memory usage in pandas
    taxi_dtypes = {
        'VendorID': pd.Int64Dtype(),
        'RatecodeID':pd.Int64Dtype(),
        'store_and_fwd_flag':str,
        'PULocationID':pd.Int64Dtype(),
        'DOLocationID':pd.Int64Dtype(),
        'passenger_count': pd.Int64Dtype(),
        'trip_distance': float,
        'payment_type': pd.Int64Dtype(),
        'fare_amount': float,
        'extra':float,
        'mta_tax':float,
        'tip_amount':float,
        'tolls_amount':float,
        'ehail_fee': float,
        'improvement_surcharge':float,
        'total_amount': float,
        'payment_type': pd.Int64Dtype(),
        'trip_type': float,
        'congestion_surcharge':float
    }

    data = pd.DataFrame()
    for month in months:
        url = os.path.join(
            url_download,
            f"green_tripdata_{year}-{month}.csv.gz"
        )
        print(url)
        data_month = pd.read_csv(
            url, sep=',', compression='gzip', dtype=taxi_dtypes, 
            parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime']
        )
        data = pd.concat([data, data_month], ignore_index=True)
        print(data.shape)
    return data
```

Add transformer
1. `+ Transformer > Python > Generic (no template)` rename to `transform_drop_rows` -> `Save and add`
```python	
@transformer
def transform(data, *args, **kwargs):
    indices = ~(data['passenger_count'].eq(0) | data['trip_distance'].eq(0))
    print(data.shape)
    data = data[indices]
    print(data.shape)
    return data
```

Add transformer adding date column
1. `+ Transformer > Python > Generic (no template)` rename to `transformer_add_date_col` -> `Save and add`
```python
@transformer
def transform(data, *args, **kwargs):
    data['lpep_pickup_date'] = data['lpep_pickup_datetime'].dt.date
    return data
```

Add transformer for retrieving unique `VendorID` values
1. `+ Transformer > Python > Generic (no template)` rename to `transform_check_vendorid` -> `Save and add`
```python
@transformer
def transform(data, *args, **kwargs):
    print(data['VendorID'].unique())
    return data
```

Add transformer for renaming columns from Camel Case to Snake Case
1. `+ Transformer > Python > Generic (no template)` rename to `transform_rename_cols` -> `Save and add`
```python
@transformer
def transform(data, *args, **kwargs):
    data.columns = data.columns.str.lower()
    return data
```

### Questions

## Question 1. Data Loading

Once the dataset is loaded, what's the shape of the data?

* **266,855 rows x 20 columns**
* 544,898 rows x 18 columns
* 544,898 rows x 20 columns
* 133,744 rows x 20 columns


### Answer 1

`266,855 rows x 20 columns`

---

## Question 2. Data Transformation

Upon filtering the dataset where the passenger count is greater than 0 _and_ the trip distance is greater than zero, how many rows are left?

* 544,897 rows
* 266,855 rows
* **139,370 rows**
* 266,856 rows

### Answer 2

`139,370 rows`

---

## Question 3. Data Transformation

Which of the following creates a new column `lpep_pickup_date` by converting `lpep_pickup_datetime` to a date?

* `data = data['lpep_pickup_datetime'].date`
* `data('lpep_pickup_date') = data['lpep_pickup_datetime'].date`
* **`data['lpep_pickup_date'] = data['lpep_pickup_datetime'].dt.date`**
* `data['lpep_pickup_date'] = data['lpep_pickup_datetime'].dt().date()`

### Answer 3

`data['lpep_pickup_date'] = data['lpep_pickup_datetime'].dt.date`

---

## Question 4. Data Transformation

What are the existing values of `VendorID` in the dataset?

* 1, 2, or 3
* **1 or 2**
* 1, 2, 3, 4
* 1

### Answer 4

`1 or 2`

---

## Question 5. Data Transformation

How many columns need to be renamed to snake case?

* 3
* 6
* 2
* **4**

### Answer 5

`4`

---

## Question 6. Data Exporting

Once exported, how many partitions (folders) are present in Google Cloud?

* **96**
* 56
* 67
* 108

### Answer 6

`95`

## Submitting the solutions

* Form for submitting: https://courses.datatalks.club/de-zoomcamp-2024/homework/hw2
* Check the link above to see the due date
  
## Solution

Will be added after the due date